In [26]:
import pandas as pd
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser

In [27]:
df = pd.read_csv('Tickers.csv', encoding = "utf-8", na_values="\\N") 

In [28]:
df['Name'] = df['Name'].str.upper()

In [29]:
df_kek = pd.read_csv('kek.csv', encoding = "utf-8", na_values="\\N")

In [30]:
abbreviations = {
    'AG': 'AKTIENGESELLSCHAFT',
    'AVRG': 'AVERAGE',
    'BK': 'BANK',
    'BKG': 'BANKING',
    'CAP': 'CAPITAL',
    'CHEMS': 'CHEMICALS',
    'CO': 'COMPANY',
    'CONSTR': 'CONSTRUCTION',
    'CORP': 'CORPORATION',
    'CTLS': 'CONTROLS',
    'FINL': 'FINANCIAL',
    'FLA': 'FLORIDA',
    'FLTNG': 'FLOATING',
    'HLDG': 'HOLDING',
    'HLDGS': 'HOLDINGS',
    'INDL': 'INDUSTRIAL',
    'INTL': 'INTERNATIONAL',
    'LABS': 'LABORATORIES',
    'MACHS': 'MACHINES',
    'MATLS': 'MATERIALS',
    'MGMT': 'MANAGEMENT',
    'NATL': 'NATIONAL',
    'PAC': 'PACIFIC',
    'PETE': 'PETROLEUM',
    'PPTYS': 'PROPERTIES',
    'PRODS': 'PRODUCTS',
    'RLTY': 'REALTY',
    'RTE': 'RATE',
    'RY': 'RAILWAY',
    'SR': 'SENIOR',
    'SVCS': 'SERVICES',
    'TR': 'TRUST'
}

In [31]:
def replace_abbreviation(name):
    name = [abbreviations.get(word) if abbreviations.get(word) is not None else word for word in name.split()]
    name = ' '.join(name)
    return name

In [32]:
for index in range(len(df_kek)):
    df_kek['Name of Issuer'].iloc[index] = replace_abbreviation(df_kek['Name of Issuer'].iloc[index])

C:\Users\vladm\AppData\Local\Temp\ipykernel_40864\4170528891.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kek['Name of Issuer'].iloc[index] = replace_abbreviation(df_kek['Name of Issuer'].iloc[index])


In [33]:
for index in range(len(df_kek)):
    df['Name'].iloc[index] = replace_abbreviation(df['Name'].iloc[index])

In [34]:
schema = Schema(Ticker = TEXT(stored=True), Name=TEXT(stored=True))

In [35]:
ix = create_in(".", schema)

In [36]:
writer = ix.writer()

In [37]:
for index, row in df.iterrows():
    writer.add_document(Ticker = str(df.Ticker.iloc[index]),
                        Name   = str(df.Name.iloc[index])
                        )
writer.commit()

In [38]:
def Search_Ticker(query):
   
    searcher = ix.searcher()
    proposed_time_strings = []
    with ix.searcher() as searcher:
        parser = QueryParser("Name", ix.schema)
        querystring = str(query)
        # querystring = u"Apple"
        myquery = parser.parse(querystring)
        print(myquery)
        results = searcher.search(myquery)
        results.fragmenter.surround = 500
        results.fragmenter.maxchars = 1500
        # print(len(results))
        if (len(results) >= 1):
            print("Ticker: " + results[0]['Ticker'])
            print("__________________________________________________")
            return results[0]['Ticker']
        else:
            print("Not match found")
            print("__________________________________________________")
            pass

In [39]:
for index in range(len(df_kek)):
    df_kek['Ticker of Issuer'].iloc[index] = Search_Ticker(df_kek['Name of Issuer'].iloc[index])

(Name:a10 AND Name:networks AND Name:inc)
Ticker: ATEN
__________________________________________________
(Name:abbvie AND Name:inc)
Ticker: ABBV
__________________________________________________


C:\Users\vladm\AppData\Local\Temp\ipykernel_40864\1003497283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kek['Ticker of Issuer'].iloc[index] = Search_Ticker(df_kek['Name of Issuer'].iloc[index])


(Name:accel AND Name:entertainment AND Name:inc)
Ticker: ACEL
__________________________________________________
(Name:adtran AND Name:inc)
Ticker: ADTN
__________________________________________________
(Name:ameresco AND Name:inc)
Ticker: AMRC
__________________________________________________
(Name:american AND Name:vanguard AND Name:corporation)
Ticker: AVD
__________________________________________________
(Name:ameris AND Name:bancorp)
Ticker: ABCB
__________________________________________________
(Name:ammo AND Name:inc)
Ticker: RTRO
__________________________________________________
(Name:ani AND Name:pharmaceuticals AND Name:inc)
Ticker: ANIP
__________________________________________________
(Name:apollo AND Name:endosurgery AND Name:inc)
Ticker: APEN
__________________________________________________
(Name:arcutis AND Name:biotherapeutics AND Name:inc)
Ticker: ARQT
__________________________________________________
(Name:argo AND Name:group AND Name:international AND Name:h

In [40]:
df_kek.head(5)

,Unnamed: 0,CIK,Name of fund,Quater of report,Signing date,Name of Issuer,Ticker of Issuer,CUSIP,Value (x$1000),Shares,Shares type,Investment Discretion,Voting Sole,Shared,None
0,0,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,A10 NETWORKS INC,ATEN,002121101,188585,10041,SH,SOLE,10041,0,0
1,1,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ABBVIE INC,ABBV,00287Y109,140,1340,SH,SOLE,1340,0,0
2,2,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ACCEL ENTERTAINMENT INC,ACEL,00436Q106,217,287328,SH,SOLE,287328,0,0
3,3,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ADTRAN INC,ADTN,00738A106,3500,187092,SH,SOLE,187092,0,0
4,4,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,AMERESCO INC,AMRC,02361E108,3452,10401,SH,SOLE,10401,0,0


In [41]:
df_kek.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             108 non-null    int64 
 1   CIK                    108 non-null    int64 
 2   Name of fund           108 non-null    object
 3   Quater of report       108 non-null    object
 4   Signing date           108 non-null    object
 5   Name of Issuer         108 non-null    object
 6   Ticker of Issuer       85 non-null     object
 7   CUSIP                  108 non-null    object
 8   Value (x$1000)         108 non-null    int64 
 9   Shares                 108 non-null    int64 
 10  Shares type            108 non-null    object
 11  Investment Discretion  108 non-null    object
 12  Voting Sole            108 non-null    int64 
 13  Shared                 108 non-null    int64 
 14  None                   108 non-null    int64 
dtypes: int64(7), object(8)


In [42]:
df_kek.to_csv(r'D:\GitHub\Yahoo-project\export.csv', index = False, header=True)